# 지난 시간 복습

1. Backward 연산을 재귀 => 반복
2. Function subclass의 사용을 편리하게
3. Code 작동 Test 기법: UnitTest module
4. 입력 크기 확장  
  
2,3,4는 거의 정리할 내용이 없음.  
1은 CS 지식 정리할 겸, 조금 자세히 준비해서 PPT 만들었음.  
자세한 내용과 꼬리 재귀에 관한 내용은 notion에 정리.  

In [76]:
import numpy as np

def as_array(x):
    return np.array(x)

class Variable:
    def __init__(self, data):
        self.x = data
        self.creator = None
        self.dx = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.dx is None:
            self.dx = np.ones_like(self.x)
        funcs = [self.creator]
        while funcs:
            func = funcs.pop()
            cur_dx = func.backward()
            prev_dx = func.output.dx
            func.input.dx = cur_dx * prev_dx
            print(func.input.dx)
            if func.input.creator is not None:
                funcs.append(func.input.creator)
        return
    
class Function:
    def __call__(self, vars):
        if type(vars) == list:
            xs = [var.x for var in vars]
            ys = list(map(Variable, map(as_array, self.forward(xs))))
            for y in ys:
                y.set_creator(self)
        else:
            xs = vars.x
            ys = as_array(self.forward(xs))
            ys = Variable(ys)
            ys.set_creator(self)
        self.input = vars
        self.output = ys
        return ys
    
    def forward(self):
        return NotImplementedError()
    
    def backward(self):
        return NotImplementedError()
    
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self):
        return np.exp(self.input.x)
    
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self):
        return 2 * self.input.x
    
def exp(x=None):
    return Exp()(x) if x is not None else Exp()

def square(x=None):
    return Square()(x) if x is not None else Square()    

class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return y,

In [77]:
x = Variable(np.array(0.5))

y = square(exp(square(x)))
y.backward()
x.dx

2.568050833375483
3.297442541400256
3.297442541400256


3.297442541400256

In [78]:
arr = np.array([0.1,0.2])
len(arr)

2

In [79]:
xs = [Variable(np.array(2.0)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.x)

5.0


# 이번주에는 총 3가지를 배운다
1. 가변 길이 인수(step11의 개선 편)
2. 가변 길이 인수(역전파)
3. 같은 변수 반복 사용

# Not Clone coding

In [95]:
from typing import List, Tuple

class Variable:
    def __init__(self, data):
        self.data = data
        self.creator = None
        self.grad = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            func = funcs.pop()
            x, y = func.input, func.output
            gy = y.grad
            gx = func.backward(gy)
            x.grad = gx
            if x.creator is not None:
                funcs.append(x.creator)
    
    def cleangrad(self):
        self.grad = None

class Function:
    def __call__(self, *inputs: List[Variable]):
        xs:List[np.array] = [input.data for input in inputs]
        ys:Tuple[np.array] = self.forward(*xs)
        
        if not isinstance(ys, tuple):
            ys = ys,
        for y in ys:
            y = Variable(y)
            y.set_creator(self)
        self.inputs:List[Variable] = inputs
        self.outputs:List[Variable] = ys
        return ys
    
    def forward(self):
        return NotImplementedError()
    
    def backward(self):
        return NotImplementedError()
    
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y,

    def backward(self, gy):
        return gy, gy
    
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx
    
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx
    

        

# 1. 가변 길이 인수(step11의 개선 편)

In [81]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Variable(object):
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

class Function(object):
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self):
        return NotImplementedError()
    
    def backward(self):
        return NotImplementedError()
    
class Square(Function):
    def forward(self, data):
        return data ** 2
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, data):
        return np.exp(data)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return (y, )
    
def add(x0, x1):
    return Add()(x0, x1)

In [82]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


# 2. 가변 길이 인수(역전파)

In [83]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y,

    def backward(self, gy):
        return gy, gy
    
class Variable(object):
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = gxs,
            
            for x, gx in zip(f.inputs, gxs):
                x.grad = gx
                if x.creator is not None:
                    funcs.append(x.creator)

class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx
    
def square(x):
    return Square()(x)

In [84]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


# 3. 같은 변수 반복 사용

아래 cell에서 x.grad는 2가 되어야 한다.  
$y = x + x = 2x$ 니까, $y' = 2$  
그런데, 우리는 grad를 안 더하고, 그냥 덮어씌우니까, 1이 되어버림.  
  
그리고 += 같은 복합대입연산자의 특징도 알려줌.  
그냥 = + 는 새로운 객체를 생성.  
**_복합연산자는 해당 객체에 그대로 덮어씌움._**  
  
즉, 최초에 x.grad = gx.  
이후에는 x.grad += gx 해버리면, x.grad랑 gx가 전부 같은 객체가 됨.  
그러면, y.grad와 x.grad가 같아져버리겠지.  
  
그런데, x.grad = x.grad + gx하면,
x.grad와 gx가 다른 객체(서로 다른 주소에 저장됨)라서,  
y.grad와 x.grad는 독립적으로 연산됨.  
  
혹시나 했는데, **_Mutable과 Immutable의 작동방식이 다름._**  
Mutable에는 위 이론이 맞음.(복합 연산자를 쓰면 같은 객체가 된다.)  
Immutable은 해당사항 없음.

In [85]:
a = 1
b = 3
print(f"a's id: {id(a)}")
print(f"b's id: {id(b)}")
a += b
print(f"a's id: {id(a)}")
a = a + b
print(f"a's id: {id(a)}")
print("#"*30)
a = [1]
b = [3]
print(f"a's id: {id(a)}")
print(f"b's id: {id(b)}")
a += b
print(f"a's id: {id(a)}")
a = a + b
print(f"a's id: {id(a)}")

a's id: 9825960
b's id: 9826024
a's id: 9826056
a's id: 9826152
##############################
a's id: 140388517687872
b's id: 140389937482240
a's id: 140388517687872
a's id: 140388514060480


In [86]:
x = Variable(np.array(3.0))
y = add(x,x)
print(f"y {y.data}")

y.backward()
print(f"x.grad {x.grad}")

y 6.0
x.grad 1.0


In [89]:
class Variable(object):
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = gxs,
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                if x.creator is not None:
                    funcs.append(x.creator)

    def cleangrad(self):
        self.grad = None

이제 grad를 더하는데, 역전파가 된 변수를 다시 사용하면, 거기에 **_또_** 더한다.  
초기화가 필요.  

In [90]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

y = add(add(x,x),x)
y.backward()
print(x.grad)

2.0
5.0


In [91]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

x.cleangrad()  
y = add(add(x,x),x)
y.backward()
print(x.grad)

2.0
3.0
